In [1]:
import pandas as pd
import ast

In [2]:
# Lista para almacenar los diccionarios JSON de cada línea
data_list = []

# Ruta del archivo JSON
file_path = 'australian_users_items.json'

# Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

# Crear un DataFrame a partir de la lista de diccionarios
data_it = pd.DataFrame(data_list)

In [3]:
# se desanidan los datos de la columna ['items']
data_it1 = data_it.explode(['items'])
data_it2 = pd.json_normalize(data_it1['items']).set_index(data_it1['items'].index)
data_it3= pd.concat([data_it2, data_it1], axis=1)

#se eliminan las columnas que no se consideran relevantes para el posterior analisis
data_it3.drop(['items_count','user_url','item_name','steam_id'], axis=1, inplace=True)

# se eliminan los datos nulos por columnas
data_it3 = data_it3.dropna(subset=['item_id'])
data_it3 = data_it3.dropna(subset=['playtime_forever'])

# se eliminan 2 columnas adicionales
data_it3.drop(['items','playtime_2weeks'], axis=1, inplace=True)

# se define el dataframe solo para aquellos juegos donde tuvieron un tiempo jugado (condicion)
data_it3 = data_it3[data_it3['playtime_forever'] != 0]

# conservar las 3 columnas mas importantes para el analisis
data_it3 = data_it3[['user_id','item_id','playtime_forever']]

# se eliminan los duplicados y valores nulos en caso de haber
data_it3 = data_it3.drop_duplicates()
data_it3 = data_it3.dropna()

# se exporta la tabla como archivo separa por comas de nombre 'user_items'
data_it3.to_csv('user_items.csv', index=False)